In [29]:
import os
import random
import shutil

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential

print(tf.__version__)

2.12.0-rc1


In [30]:
# initial labeling

# train_images = os.listdir('screens_training')
# train_images = [t for t in train_images if '.jpg' in t]

# for (name, val) in zip(train_images, y):
#     if val == 1:
#         shutil.move(f'screens_training\\{name}', f'screens_training\\True\\{name}')
#     elif val == 0:
#         shutil.move(f'screens_training\\{name}', f'screens_training\\False\\{name}')


In [31]:
image_size = (256, 256)
train_ds = tf.keras.utils.image_dataset_from_directory(directory='screens_training_balanced', 
                                                       image_size=image_size,
                                                       label_mode='binary',
                                                       seed=420,
                                                       shuffle=True,
                                                       subset='training',
                                                       validation_split=0.2)

Found 18818 files belonging to 2 classes.
Using 15055 files for training.


In [32]:
validation_ds = tf.keras.utils.image_dataset_from_directory(directory='screens_training_balanced', 
                                                            image_size=image_size,
                                                            label_mode='binary',
                                                            seed=420,
                                                            shuffle=True,
                                                            subset='validation',
                                                            validation_split=0.2)

Found 18818 files belonging to 2 classes.
Using 3763 files for validation.


In [33]:
# sanity check the sets have no common part
for name in validation_ds.file_paths:
    if name in train_ds.file_paths:
        print('error')

In [34]:
with open('validation_ds.txt', 'w') as f:
    for name in validation_ds.file_paths:
        f.write(f'{name[-9:-4]}\n')

In [35]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
validation_ds = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [36]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(256, 256, 3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [37]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

In [38]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [43]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=METRICS)

In [40]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 rescaling_2 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 256, 256, 16)      448       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 128, 128, 16)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 128, 128, 32)      4640      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                  

In [44]:
model.fit(train_ds, 
          epochs=2,
          validation_data=validation_ds)

Epoch 1/2
118/471 [======>.......................] - ETA: 5:57 - loss: 0.6912 - tp: 1498.0000 - fp: 1248.0000 - tn: 2543.0000 - fn: 2250.0000 - accuracy: 0.5360 - precision: 0.5455 - recall: 0.3997 - auc: 0.5473 - prc: 0.5369

KeyboardInterrupt: 

In [42]:
model.save('model5')

INFO:tensorflow:Assets written to: model4\assets


INFO:tensorflow:Assets written to: model4\assets


In [27]:
# test_ds = tf.keras.utils.image_dataset_from_directory(directory='screens_10001_11000_unlabeled', 
#                                                        image_size=image_size, 
#                                                       labels=None,
#                                                      shuffle=False) # ważne

Found 1000 files belonging to 1 classes.


In [33]:
# test_ds = tf.keras.utils.image_dataset_from_directory(directory='screens_40001_50000_unlabeled', 
#                                                        image_size=image_size, 
#                                                       labels=None,
#                                                      shuffle=False) # ważne

Found 10000 files belonging to 1 classes.


In [34]:
# predictions = model.predict(test_ds)

313/313 [==============================] - 126s 403ms/step


In [35]:
# game_states_data_training = pd.read_csv('game_states_data_training.csv')

# X = game_states_data_training
# X = X[X['instance_id'] >= 40001]
# y_test = X['frag'].astype(int).values

# y_pred = predictions

# print(f'\nTest score: {roc_auc_score(y_test, y_pred)}')


Test score: 0.6225310757823442


In [36]:
# with open('img_pred_40001_50000.txt', 'w') as f:
#     for y in y_pred:
#         f.write(f'{y[0]}\n')

In [3]:
# # RUN IT ON DEMAND ONLY

# false_training_images_to_move = os.listdir('screens_training/False')
# i = 0
# no_files = 9409
# while i < no_files:
#     file = random.choice(false_training_images_to_move)
#     shutil.copyfile(f'screens_training/False/{file}', f'screens_training_balanced/False/{file}')
#     false_training_images_to_move.remove(file)
#     i += 1

In [32]:
# false_training_images_to_move = os.listdir('screens_training/False')
# for file in false_training_images_to_move:
#     if int(file[:5]) >= 40000:
#         shutil.copyfile(f'screens_training/False/{file}', f'screens_40001_50000_unlabeled/{file}')

In [13]:
# saved_model = tf.keras.models.load_model('model3')

# # Check its architecture
# saved_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 256, 256, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 128, 128, 16)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 128, 128, 32)      4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 64, 64, 64)       

In [14]:
# # official testset predictions

# test_ds = tf.keras.utils.image_dataset_from_directory(directory='screens_test', 
#                                                        image_size=image_size, 
#                                                       labels=None,
#                                                      shuffle=False) # ważne

# y_pred = saved_model.predict(test_ds)

# with open('img_screens_test_res.txt', 'w') as f:
#     for y in y_pred:
#         f.write(f'{y[0]}\n')

Found 10000 files belonging to 1 classes.
313/313 [==============================] - 149s 474ms/step


In [10]:
# test_ds = tf.keras.utils.image_dataset_from_directory(directory='screens_40001_50000_unlabeled', 
#                                                        image_size=image_size, 
#                                                       labels=None,
#                                                      shuffle=False) # ważne

# predictions = saved_model.predict(test_ds)

# game_states_data_training = pd.read_csv('game_states_data_training.csv')

# X = game_states_data_training
# X = X[X['instance_id'] >= 40001]
# y_test = X['frag'].astype(int).values

# y_pred = predictions

# print(f'\nTest score: {roc_auc_score(y_test, y_pred)}')

Found 10000 files belonging to 1 classes.
313/313 [==============================] - 129s 409ms/step

Test score: 0.6225310757823442


In [11]:
# saved_model.fit(train_ds, 
#                 epochs=10,
#                 validation_data=validation_ds)

Epoch 1/10
373/373 [==============================] - 483s 1s/step - loss: 0.5704 - tp: 4082.0000 - fp: 1821.0000 - tn: 4162.0000 - fn: 1870.0000 - accuracy: 0.6907 - precision: 0.6915 - recall: 0.6858 - auc: 0.7664 - prc: 0.7618 - val_loss: 0.7059 - val_tp: 822.0000 - val_fp: 606.0000 - val_tn: 870.0000 - val_fn: 685.0000 - val_accuracy: 0.5672 - val_precision: 0.5756 - val_recall: 0.5455 - val_auc: 0.6103 - val_prc: 0.6127
Epoch 2/10
373/373 [==============================] - 367s 982ms/step - loss: 0.5528 - tp: 4170.0000 - fp: 1745.0000 - tn: 4238.0000 - fn: 1782.0000 - accuracy: 0.7045 - precision: 0.7050 - recall: 0.7006 - auc: 0.7856 - prc: 0.7807 - val_loss: 0.7161 - val_tp: 811.0000 - val_fp: 596.0000 - val_tn: 880.0000 - val_fn: 696.0000 - val_accuracy: 0.5669 - val_precision: 0.5764 - val_recall: 0.5382 - val_auc: 0.6098 - val_prc: 0.6075
Epoch 3/10
373/373 [==============================] - 361s 966ms/step - loss: 0.5403 - tp: 4229.0000 - fp: 1674.0000 - tn: 4309.0000 - fn: 

In [12]:
# test_ds = tf.keras.utils.image_dataset_from_directory(directory='screens_40001_50000_unlabeled', 
#                                                        image_size=image_size, 
#                                                       labels=None,
#                                                      shuffle=False) # ważne

# predictions = saved_model.predict(test_ds)

# game_states_data_training = pd.read_csv('game_states_data_training.csv')

# X = game_states_data_training
# X = X[X['instance_id'] >= 40001]
# y_test = X['frag'].astype(int).values

# y_pred = predictions

# print(f'\nTest score: {roc_auc_score(y_test, y_pred)}')

Found 10000 files belonging to 1 classes.
313/313 [==============================] - 126s 402ms/step

Test score: 0.6124977766482134
